# Data preparation
The first phase is meant to **explore** and **prepare** the dataset for the reinforcement learning model.
It's crucial to have a **clean** and **well-structured** dataset to make a good reinforcement learning model.

---
## Loading data

In [13]:
import numpy as np
import pandas as pd

file_path = 'datasets/reinforcement_learning_export.csv'
df = pd.read_csv(file_path, sep = ';')
df.head()

,Currency,Date,Open,High,Low,Close,Volume,Market Cap
0,ripple,24/04/19,"0,321114","0,321282","0,296982","0,302318",1.517791e+09,12698877293
1,ripple,23/04/19,"0,323844","0,328396","0,320919","0,321222",1.077334e+09,13492933875
2,ripple,22/04/19,"0,322277","0,32935","0,320237","0,323934",1.131094e+09,13606823301
3,ripple,21/04/19,"0,328678","0,329627","0,318746","0,322449",1.005804e+09,13533407430
4,ripple,20/04/19,"0,331871","0,333213","0,324969","0,328476",9.315708e+08,13786384592


---
## Exploring data
Before the **cleaning** and **preprocessing** phases, we'll explore the dataset looking for patterns or anomalies.

In [11]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13576 entries, 0 to 13575
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Currency    13576 non-null  object 
 1   Date        13576 non-null  object 
 2   Open        13576 non-null  object 
 3   High        13576 non-null  object 
 4   Low         13576 non-null  object 
 5   Close       13576 non-null  object 
 6   Volume      12945 non-null  float64
 7   Market Cap  13564 non-null  object 
dtypes: float64(1), object(7)
memory usage: 848.6+ KB


,Volume
count,1.294500e+04
mean,7.697894e+08
std,1.954981e+09
min,0.000000e+00
25%,1.842150e+06
50%,4.762450e+07
75%,4.768040e+08
max,2.655755e+10


---
# Data cleaning
We need to convert columns into the right data types and remove missing values.

In [35]:
df['Currency'] = df['Currency'].astype('string')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

df['Volume'] = df['Volume'].astype('string')
df['Market Cap'] = df['Market Cap'].astype('string')
df['Open'] = df['Open'].astype('string')
df['High'] = df['High'].astype('string')
df['Low'] = df['Low'].astype('string')
df['Close'] = df['Close'].astype('string')

In [36]:
df.set_index('Date', inplace=True)

ref: [https://gym-trading-env.readthedocs.io/en/latest/index.html](https://gym-trading-env.readthedocs.io/en/latest/index.html)

---
# Import dataset
In order for this enviroment to work, the dataset must be in the following format:
- it must be ordered by **ascending date**
- the index must be a **DatetimeIndex**
- it must have **Close**, **Open**, **High**, **Low**, **Volume** labels at least

In [39]:
import pandas as pd
import gymnasium as gym
import gym_trading_env

In [40]:
dataset = './datasets/BTC_USD-Hourly.csv'
df = pd.read_csv(dataset, parse_dates = ['date'], index_col = 'date')
df.sort_index(inplace = True)
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)

# Adding more features
The reinforcement learning agent will need inputs.
The enviroment recognizes as input every column that has **feature** in its name.

In [43]:
df['feature_close'] = df['close'].pct_change()
df['feature_open'] = df['open'] / df['close']
df['feature_high'] = df['high'] / df['close']
df['feature_low'] = df['low'] / df['close']
df['feature_volume'] = df['Volume USD'] / df['Volume USD'].rolling(7 * 24).max()

df.dropna(inplace = True)

# Creating the enviroment
Let's examine the parameters of the enviroment:
- **name**: the name of the enviroment
- **df**: the dataframe
- **positions**: list of the positions allowed by the enviroment
- **trading_fees**: the trading fees (buy and sell operations)
- **borrow_interest_rate**: the interest rate for borrowing money

In [44]:
env = gym.make(
    'TradingEnv',
    name = 'BTCUSD',
    df = df,
    positions = [-1, 0, 1],
    trading_fees = 0.01 / 100,
    borrow_interest_rate = 0.003 / 100
)

# Run the enviroment

In [45]:
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    position_index = env.action_space.sample()
    observation, reward, done, truncated, info = env.step(position_index)

Market Return : 507.09%   |   Portfolio Return : -92.77%   |   
